# Introduction to Machine Learning with Football Players

This notebook is an introduction to machine learning applied on [this](https://github.com/loistal/ADA_labs/blob/master/04%20-%20Applied%20ML/DATA.md) dataset.

The goal is to be able to predict the skin color of a player, using supervised and unsupervised techniques.

## Data Pre-Processing

First, we need to work on the data to clean it and get it ready for the learning part.

We use [this](http://nbviewer.jupyter.org/github/mathewzilla/redcard/blob/master/Crowdstorming_visualisation.ipynb) notebook to help us achieving this task.